In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None;
    try:
        conn = sqlite3.connect(':memory:')
        print('loaded sqlite3 version =',sqlite3.version)
    except Error as e:
        print(e)
    return conn

In [3]:
conn = create_connection()
curr = conn.cursor()

loaded sqlite3 version = 2.6.0


In [4]:
# load the SQL ddf from PHPMyAdmin export
# __NOTE__: I needed to remove the string 'ENGINE=InnoDB DEFAULT CHARSET=utf8' from 
# each create table statement in order to get data to load.
sql_file = open("country-club-db.sql")
sql_as_string = sql_file.read()
curr.executescript(sql_as_string)

In [5]:
# this is an example of executing a query directly on database
for row in curr.execute("SELECT firstname FROM Members where firstname = 'Tracy';"):
    print(row)

('Tracy',)


In [6]:
# import Facilites table into pandas
facilities_df = pd.DataFrame()
facilities_df = pd.read_sql_query("SELECT * FROM Facilities;", conn)
facilities_df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [7]:
# import Bookings table into pandas
bookings_df = pd.DataFrame()
bookings_df = pd.read_sql_query("SELECT * FROM Bookings;", conn)
bookings_df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [8]:
# import Members table into pandas
members_df = pd.DataFrame()
members_df = pd.read_sql_query("SELECT * FROM Members;", conn)
members_df.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


### Assignment Questions

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [27]:
results = curr.execute(
"""
SELECT f.name AS facility_name,
SUM(CASE 
    WHEN b.memid = 0
    THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
END) AS total_revinue
FROM Facilities as f
INNER JOIN Bookings as b ON b.facid = f.facid
GROUP BY f.facid HAVING 
    (SUM(CASE
    WHEN b.memid = 0
    THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
    END)) > 1000
ORDER BY total_revinue DESC;
"""
)
headers = [i[0] for i in curr.description]
print(headers)
print('-'*50)
for row in results:
    print(row)

['facility_name', 'total_revinue']
--------------------------------------------------
('Massage Room 1', 50351.600000000275)
('Massage Room 2', 14454.599999999995)
('Tennis Court 2', 14310)
('Tennis Court 1', 13860)
('Squash Court', 13468.0)
('Badminton Court', 1906.5)


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [29]:
results = curr.execute(
"""
SELECT DISTINCT m1.memid, m1.firstname as member_first, m1.surname as member_last, m1.recommendedby as recommendedby_memid, m2.firstname as recommender_first, m2.surname as recommender_last
FROM Members as m1
LEFT JOIN Members as m2 on m1.recommendedby = m2.memid where m1.recommendedby <> ''
ORDER BY m1.surname, m1.firstname, m2.surname, m2.firstname;
""")
headers = [i[0] for i in curr.description]
print(headers)
print('-'*80)
for row in results:
    print(row)


['memid', 'member_first', 'member_last', 'recommendedby_memid', 'recommender_first', 'recommender_last']
--------------------------------------------------------------------------------
(15, 'Florence', 'Bader', '9', 'Ponder', 'Stibbons')
(12, 'Anne', 'Baker', '9', 'Ponder', 'Stibbons')
(16, 'Timothy', 'Baker', '13', 'Jemima', 'Farrell')
(8, 'Tim', 'Boothe', '3', 'Tim', 'Rownam')
(5, 'Gerald', 'Butters', '1', 'Darren', 'Smith')
(22, 'Joan', 'Coplin', '16', 'Timothy', 'Baker')
(36, 'Erica', 'Crumpet', '2', 'Tracy', 'Smith')
(7, 'Nancy', 'Dare', '4', 'Janice', 'Joplette')
(20, 'Matthew', 'Genting', '5', 'Gerald', 'Butters')
(35, 'John', 'Hunt', '30', 'Millicent', 'Purview')
(11, 'David', 'Jones', '4', 'Janice', 'Joplette')
(26, 'Douglas', 'Jones', '11', 'David', 'Jones')
(4, 'Janice', 'Joplette', '1', 'Darren', 'Smith')
(21, 'Anna', 'Mackenzie', '1', 'Darren', 'Smith')
(10, 'Charles', 'Owen', '1', 'Darren', 'Smith')
(17, 'David', 'Pinker', '13', 'Jemima', 'Farrell')
(30, 'Millicent', 'Pu

Q12: Find the facilities with their usage by member, but not guests 

In [30]:
results = curr.execute(
"""
SELECT DISTINCT
f.name as facility,
SUM(CASE 
    WHEN b.memid = 0
    THEN 0
    ELSE b.slots
END) AS facility_usage
FROM Bookings as b
INNER JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid
ORDER BY facility_usage DESC
""")
headers = [i[0] for i in curr.description]
print(headers)
print('-'*80)
for row in results:
    print(row)

['facility', 'facility_usage']
--------------------------------------------------------------------------------
('Badminton Court', 1086)
('Tennis Court 1', 957)
('Massage Room 1', 884)
('Tennis Court 2', 882)
('Snooker Table', 860)
('Pool Table', 856)
('Table Tennis', 794)
('Squash Court', 418)
('Massage Room 2', 54)


Q13: Find the facilities usage by month, but not guests

In [35]:
results = curr.execute(
"""
SELECT DISTINCT
f.name as facility,
(strftime('%m', b.starttime)) AS month, 
SUM(CASE 
    WHEN b.memid = 0
    THEN 0
    ELSE b.slots
END) AS monthly_facility_usage
FROM Bookings as b
INNER JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid, month
ORDER BY facility, month
""")
headers = [i[0] for i in curr.description]
print(headers)
print('-'*80)
for row in results:
    print(row)

['facility', 'month', 'monthly_facility_usage']
--------------------------------------------------------------------------------
('Badminton Court', '07', 165)
('Badminton Court', '08', 414)
('Badminton Court', '09', 507)
('Massage Room 1', '07', 166)
('Massage Room 1', '08', 316)
('Massage Room 1', '09', 402)
('Massage Room 2', '07', 8)
('Massage Room 2', '08', 18)
('Massage Room 2', '09', 28)
('Pool Table', '07', 110)
('Pool Table', '08', 303)
('Pool Table', '09', 443)
('Snooker Table', '07', 140)
('Snooker Table', '08', 316)
('Snooker Table', '09', 404)
('Squash Court', '07', 50)
('Squash Court', '08', 184)
('Squash Court', '09', 184)
('Table Tennis', '07', 98)
('Table Tennis', '08', 296)
('Table Tennis', '09', 400)
('Tennis Court 1', '07', 201)
('Tennis Court 1', '08', 339)
('Tennis Court 1', '09', 417)
('Tennis Court 2', '07', 123)
('Tennis Court 2', '08', 345)
('Tennis Court 2', '09', 414)


In [ ]:
# Be sure to close the connection
curr.close()
conn.close()